In [ ]:
!git clone https://github.com/Kemsekov/kemsekov_torch
!cd kemsekov_torch && git pull

In [ ]:
segmentation_model = 'vlad/seg-drone.pt'
yolo_model = "andry/best2.pt"
classification="egor/best2.pth"

pad_to_multiple=128
padding_value=0.5

# трешхолд для сегментации
seg_threshold = 0.5

# трешхолд для объектов после сегментации
prob_threshold=0.6

# трешхолд для первичной классификации боксов моделью егора
classification_threshold = 0.9

# размер тайла, который вырезается для области предсказанной моделькой сегментации
tile_size=512

# нужно ли логить промежуточные результаты
log = True

device = 'cuda'

# макс расстояние между пикселями в одном кластере
dbscan_eps = 4

# мин кол-во пикселей в кластере
dbscan_min_samples = 2

# трешхолд для копий детекций
copies_threshold=1e-3

from inference_code import DroneObjectDetector
detector = DroneObjectDetector(
    segmentation_model,
    yolo_model,
    classification,
    pad_to_multiple,
    padding_value,
    seg_threshold,
    prob_threshold,
    classification_threshold,
    tile_size,
    log,
    device,
    dbscan_eps,
    dbscan_min_samples,
    copies_threshold
)

In [ ]:
import os
# path to images
images = '/home/vlad/Documents/DRONE-DATASET/train/images'
labels = '/home/vlad/Documents/DRONE-DATASET/train/labels'
images_files = {f.split(".")[0]:os.path.join(images,f) for f in os.listdir(images)}
label_files = {f.split(".")[0]:os.path.join(labels,f) for f in os.listdir(labels)}
keys = set(label_files.keys()).intersection(set(images_files.keys()))
keys=list(keys)

In [ ]:
from drone_common import plot_image_with_boxes_and_probs
import pandas as pd
import random
import PIL.Image as Image

while True:
    ind = random.randint(0,len(images_files)-1)
    key = keys[ind]
    imp = images_files[key]
    lb = label_files[key]
    true_detection = pd.read_csv(lb,header=None,sep=' ', names=['class', 'x', 'y', 'w', 'h'])
    im = Image.open(imp)
    
    detection = detector.detect(im,true_detection)
    
    # if len(detection)!=0:
    if log: print(im.size)
    if log: print("-"*20)
    plot_image_with_boxes_and_probs(im,detection)
    # display(T.ToPILImage()(r(out)))
    
    # extract_detections(im,detection,imp)
    break